In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../util')
sys.path.append('../experiments')

In [4]:
from convert_annotations import read_webanno, resolve_ellipses, webanno_to_iob_df, EMPTY_REGEX
from pathlib import Path
from datetime import datetime
import pandas as pd

In [97]:
%%time
tsv_folder = '../../data_ggponc/output_ellipses/'
#tsv_folder = 'ellipses_dev/'

webanno_df, sentences = read_webanno(list(Path(tsv_folder).glob('*.tsv')))

CPU times: user 10.6 s, sys: 359 ms, total: 11 s
Wall time: 12.3 s


In [98]:
%%time

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    iob_df = webanno_to_iob_df(webanno_df, 'detail', True, debug=False, collect_errors=False, skip_errors=True, all_columns=True)
    iob_df['id'] = iob_df.entity_id.where(~iob_df.entity_id.isna(), iob_df.spec_id)

100%|█████████████████████████████████████████████████████████████████████████████| 49821/49821 [10:30<00:00, 79.02it/s]


CPU times: user 10min 55s, sys: 3.32 s, total: 10min 59s
Wall time: 10min 56s


In [99]:
# Anything containing a prefix, suffix or fragment is potentially an elliptical construct
ellipses = iob_df[~iob_df.prefix.str.match(EMPTY_REGEX) | ~iob_df.suffix.str.match(EMPTY_REGEX) | ~iob_df.fragment.str.match(EMPTY_REGEX)].copy()

In [100]:
iob_df

token_id ts_id     span  \
file                                  sentence_id                            
00_mundhoehlenkarzinom_0002.tsv       1                   1   1-1     0-19   
                                      1                   2   1-2    20-23   
                                      1                   3   1-3    24-27   
                                      1                   4   1-4    28-37   
                                      1                   5   1-5    38-43   
...                                                     ...   ...      ...   
29_adulte-weichgewebesarkome_0405.tsv 4                  26  4-26  755-756   
                                      4                  27  4-27  757-767   
                                      4                  28  4-28  768-779   
                                      4                  29  4-29  779-780   
                                      4                  30  4-30  780-781   

                                                                 token  \
file                                  sentence_id                        
00_mundhoehlenkarzinom_0002.tsv       1            Hauptrisikofaktoren   
                                      1                            für   
                                      1                            das   
                                      1                      Auftreten   
                                      1                          eines   
...                                                                ...   
29_adulte-weichgewebesarkome_0405.tsv 4                              .   
                                      4                     lipotrophe   
                                      4                    Chemikalien   
                                      4                              )   
                                      4                              .   

                                                           value  \
file                                  sentence_id                  
00_mundhoehlenkarzinom_0002.tsv       1                  Finding   
                                      1                        _   
                                      1                        _   
                                      1                        _   
                                      1                        _   
...                                                          ...   
29_adulte-weichgewebesarkome_0405.tsv 4                        _   
                                      4            Specification   
                                      4                Substance   
                                      4                        _   
                                      4                        _   

                                                               detail  \
file                                  sentence_id                       
00_mundhoehlenkarzinom_0002.tsv       1                 Other Finding   
                                      1                             _   
                                      1                             _   
                                      1                             _   
                                      1                             _   
...                                                               ...   
29_adulte-weichgewebesarkome_0405.tsv 4                             _   
                                      4                             *   
                                      4            External Substance   
                                      4                             _   
                                      4                             _   

                                                  specified_by prefix suffix  \
file                                  sentence_id                              
00_mundhoehlenkarzinom_0002.tsv       1                      _      *      *  

In [147]:
%%time
results = resolve_ellipses(ellipses, iob_df)

/mnt/nfs/home/Florian.Borchert/workspace/ggponc_annotation/notebooks/../util/convert_annotations.py:166: PerformanceWarning: indexing past lexsort depth may impact performance.
  sentence = ellipses.loc[idx]
/mnt/nfs/home/Florian.Borchert/workspace/ggponc_annotation/notebooks/../util/convert_annotations.py:168: PerformanceWarning: indexing past lexsort depth may impact performance.
  full_sentence = iob_df.loc[idx]


CPU times: user 22.6 s, sys: 35.8 ms, total: 22.6 s
Wall time: 22.6 s


In [102]:
results

,file,sentence_id,full_sentence,span_index_start,span_index_end,full_span,offsets,resolution,fragment,missing_prefix,missing_suffix
0,00_mundhoehlenkarzinom_0002.tsv,1,"[Hauptrisikofaktoren, für, das, Auftreten, ein...",7,11,"[chronischer, Tabak, -, oder, Alkoholabusus]","[[69, 80], [81, 86], [86, 87], [88, 92], [93, ...","[chronischer, Tabakabusus, , oder, Alkoholabusus]",True,False,True
1,00_mundhoehlenkarzinom_0002.tsv,2,"[Bei, chronischem, Tabak, -, oder, Alkoholabus...",1,5,"[chronischem, Tabak, -, oder, Alkoholabusus]","[[4, 15], [16, 21], [21, 22], [23, 27], [28, 41]]","[chronischem, Tabakabusus, , oder, Alkoholabusus]",True,False,True
2,00_mundhoehlenkarzinom_0002.tsv,5,"[Neben, dem, Konsum, von, Tabak, oder, Alkohol...",16,22,"[übermäßiger, Konsum, von, Fleisch, oder, gebr...","[[90, 101], [102, 108], [109, 112], [113, 120]...","[übermäßiger, Konsum, von, Fleisch, oder, von ...",False,True,False
3,00_mundhoehlenkarzinom_0005.tsv,12,"[Die, hohe, Diskrepanz, zwischen, p16, Immunhi...",8,11,"[HPV-RNA, und, HPV-DNA, Nachweis]","[[55, 62], [63, 66], [67, 74], [75, 83]]","[HPV-RNA-Nachweis, und, HPV-DNA, Nachweis]",True,False,True
4,00_mundhoehlenkarzinom_0005.tsv,18,"[Die, Prävalenz, von, HPV16, E6, oder, E7, Ant...",3,9,"[HPV16, E6, oder, E7, Antikörpern, in, Mundhöh...","[[18, 23], [24, 26], [27, 31], [32, 34], [35, ...","[HPV16 Antikörpern, E6 Antikörpern, oder, E7, ...",True,False,True
...,...,...,...,...,...,...,...,...,...,...,...
5197,29_adulte-weichgewebesarkome_0397.tsv,4,"[Sehr, viel, häufiger, wird, bei, Männern, unt...",18,22,"[Weichteiltumor, des, Skrotums, oder, Samenstr...","[[116, 130], [131, 134], [135, 143], [144, 148...","[Weichteiltumor, des, Skrotums, oder, des Same...",False,True,False
5198,29_adulte-weichgewebesarkome_0397.tsv,6,"[Klinisch, besteht, meist, eine, unilaterale, ...",4,13,"[unilaterale, ,, sich, langsam, vergrößernde, ...","[[28, 39], [39, 40], [41, 45], [46, 53], [54, ...","[unilaterale, ,, sich, langsam, vergrößernde, ...",False,True,False
5199,29_adulte-weichgewebesarkome_0403.tsv,1,"[Es, besteht, Forschungsbedarf, in, der, Kläru...",7,14,"[molekulare, Prognosemarker, für, Sarkome, ode...","[[53, 63], [64, 78], [79, 82], [83, 90], [91, ...","[molekulare, Prognosemarker, für, Sarkome, ode...",False,True,False
5200,29_adulte-weichgewebesarkome_0404.tsv,6,"[Bei, der, Zweitlinien, -, und, Folgetherapie,...",2,5,"[Zweitlinien, -, und, Folgetherapie]","[[8, 19], [19, 20], [21, 24], [25, 38]]","[Zweitlinien-therapie, , und, Folgetherapie]",True,False,True


In [148]:
sentence_idx = pd.DataFrame(sentences, columns=['raw_sentence'], index=webanno_df.index.drop_duplicates())

In [149]:
clean_results = results.merge(right=sentence_idx, left_on=['file', 'sentence_id'], right_index=True)

In [150]:
def get_full_res(row):
    sent = row.raw_sentence
    spans = row.offsets
    resolution = row.resolution
    assert len(spans) == len(resolution)
    j = 0
    for span, res in zip(spans, resolution):
        start, end = span
        start += j
        end += j
        sent = sent[:start] + res + sent[end:]
        j += (start - end + len(res))
    return sent

In [151]:
clean_results['full_resolution'] = clean_results.apply(get_full_res, axis=1)

In [160]:
import numpy as np
for _, k in clean_results[clean_results.duplicated(['file', 'sentence_id'])][['file', 'sentence_id']].iterrows():
    f = k['file']
    s = k['sentence_id']
    dups = clean_results[(clean_results.file == f) & (clean_results.sentence_id == s)]
    
    for i, inner in dups.iterrows():
        for j, outer in dups.iterrows():
            if i != j and inner.span_index_start >= outer.span_index_start and inner.span_index_end <= outer.span_index_end:
                clean_results.loc[i, 'outer'] = j

In [166]:
clean_results = clean_results[clean_results.outer.isna()]

In [168]:
ellipses_sentence_count = len(results[['file', 'sentence_id']].drop_duplicates())
sentence_count = 78090
ellipses_sentence_count, sentence_count, ellipses_sentence_count / sentence_count

(4626, 78090, 0.05923933922397234)

In [169]:
print('Ellipses', len(results))
print('Forward ellipses', len(results[results.missing_prefix & ~results.missing_suffix]), len(results[results.fragment & results.missing_prefix & ~results.missing_suffix]))
print('Backward ellipses', len(results[~results.missing_prefix & results.missing_suffix]), len(results[results.fragment & ~results.missing_prefix & results.missing_suffix]))
print('Complex ellipses', len(results[results.missing_prefix & results.missing_suffix]), len(results[results.fragment & results.missing_prefix & results.missing_suffix]))

Ellipses 5202
Forward ellipses 1945 831
Backward ellipses 2955 2718
Complex ellipses 263 180


In [109]:
import random

def show_random(df):
    i = random.randint(0, len(df))
    item = df.iloc[i]    
    assert len(item.full_span) == len(item.resolution)
    spans = []
    solutions = []
    marker = []
    for span, res in zip(item.full_span, item.resolution):
        if span == res:
            marker.append(' ' * len(span))
        else:
            marker.append('^' * len(res))
        start_index = res.find(span)
        spans.append(span.rjust(start_index + len(span)).ljust(len(res) - start_index))
        solutions.append(res)
    print(' '.join(spans))
    print(' '.join(solutions))
    print(' '.join(marker))
    print(item.file, item.sentence_id, 'fragment:', item.fragment)
    print(item.raw_sentence)
    print(item.full_resolution)
    print('...........')
    print('')

In [171]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.fragment])

Grad 3      4
Grad 3 Grad 4
       ^^^^^^
27_supportive-therapie_0730.tsv 7 fragment: True
Allerdings bleibt das Ergebnis fragwürdig, da 3% fehlende Assessments in der Placebogruppe als orale Mukositis (Grad 3 4) eingestuft wurden.
Allerdings bleibt das Ergebnis fragwürdig, da 3% fehlende Assessments in der Placebogruppe als orale Mukositis (Grad 3 Grad 4) eingestuft wurden.
...........

keratin-low versus –         high Plattenepithelkarzinome
keratin-low versus – keratin-high Plattenepithelkarzinome
                     ^^^^^^^^^^^^                        
05_zervixkarzinom_0108.tsv 50 fragment: True
Zur Relevanz molekularer Marker und der auf molekularen Untersuchungen des The Cancer Genome Atlas-Projektes beruhende TCGA-Klassifikation (keratin-low versus –high Plattenepithelkarzinome) stehen derzeit nur eingeschränkte Daten zur Verfügung [REF], [REF], [REF].
Zur Relevanz molekularer Marker und der auf molekularen Untersuchungen des The Cancer Genome Atlas-Projektes beruhende TCGA-K

In [173]:
for _ in range(0, 10):
    show_random(clean_results[~clean_results.fragment])

Zufriedenheit mit der sexuellen Aktivität ,     dem Orgasmus
Zufriedenheit mit der sexuellen Aktivität , mit dem Orgasmus
                                            ^^^^^^^         
28_komplementaermedizin_0169.tsv 66 fragment: False
Auch zeigten diese Probanden bessere Werte für die Zufriedenheit mit der sexuellen Aktivität, dem Orgasmus und dem generellen sexuellen Wohlbefinden gegenüber der Placebogruppe, welche wiederrum bessere Werte zeigte als die Placebo-Kontrollgruppe (Mittelwert(SD) = Zufriedenheit:
Auch zeigten diese Probanden bessere Werte für die Zufriedenheit mit der sexuellen Aktivität, mit dem Orgasmus und dem generellen sexuellen Wohlbefinden gegenüber der Placebogruppe, welche wiederrum bessere Werte zeigte als die Placebo-Kontrollgruppe (Mittelwert(SD) = Zufriedenheit:
...........

NCI-CTCAE Kriterien Version 2 oder         3
NCI-CTCAE Kriterien Version 2 oder Version 3
                                   ^^^^^^^^^
27_supportive-therapie_0318.tsv 3 fragment: False
Zur

In [176]:
date = datetime.now().strftime("%Y%m%d_%H%M%S")

In [177]:
date

'20220704_171728'

In [178]:
clean_results.to_excel(f'ellipses_clean_{date}.xlsx')